In [5]:
import pandas as pd
import wandb
from datetime import datetime
import numpy as np


class Results():
    def __init__(self):
        num_seeds = 5
        columns = [f"nguyen_{i}" for i in range(1, 13, 1)]
        self.result_dict = {}
        for target in ['num_states_to_perfect_fit_test', 'time to perfect fit', 'num simulation to perfect fit']:
            self.result_dict[target] = {}
            self.result_dict[target]['results_grammar_1_Normal'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_uniform_1_Normal'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_grammar_1_AmEx'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_uniform_1_AmEx'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)

            self.result_dict[target]['results_grammar_2_Normal'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_uniform_2_Normal'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_grammar_2_AmEx'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
            self.result_dict[target]['results_uniform_2_AmEx'] = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)

    def fill_results(self, config, summary):
        seed = config['seed']
        data_path = config['data_path']
        engine = config['MCTS_engine']
        prior = config['prior_source']
        dataset = config['data_path'].split('/')[1]
        for target in ['num_states_to_perfect_fit_test', 'time to perfect fit', 'num simulation to perfect fit']:
            value = summary[target]
            if 'data_grammar_1' in data_path and engine == 'Endgame' and prior == 'grammar':
                self.result_dict[target]['results_grammar_1_AmEx'].loc[seed, dataset] = value
            elif 'data_grammar_1' in data_path and engine == 'Normal' and prior == 'grammar':
                self.result_dict[target]['results_grammar_1_Normal'].loc[seed, dataset] = value
            elif 'data_grammar_1' in data_path and engine == 'Endgame' and prior == 'uniform':
                self.result_dict[target]['results_uniform_1_AmEx'].loc[seed, dataset] = value
            elif 'data_grammar_1' in data_path and engine == 'Normal' and prior == 'uniform':
                self.result_dict[target]['results_uniform_1_Normal'].loc[seed, dataset] = value

            elif 'data_grammar_2' in data_path and engine == 'Endgame' and prior == 'grammar':
                self.result_dict[target]['results_grammar_2_AmEx'].loc[seed, dataset] = value
            elif 'data_grammar_2' in data_path and engine == 'Normal' and prior == 'grammar':
                self.result_dict[target]['results_grammar_2_Normal'].loc[seed, dataset] = value
            elif 'data_grammar_2' in data_path and engine == 'Endgame' and prior == 'uniform':
                self.result_dict[target]['results_uniform_2_AmEx'].loc[seed, dataset] = value
            elif 'data_grammar_2' in data_path and engine == 'Normal' and prior == 'uniform':
                self.result_dict[target]['results_uniform_2_Normal'].loc[seed, dataset] = value


api = wandb.Api()
entity, project = "jgu-wandb", "neural_guided_symbolic_regression_final"
# Example: January 1st, 2024

# created_at (str): ISO timestamp when the run was started
runs = api.runs(entity + "/" + project, filters={"tags": {"$in": ["no_net"]}})

summary_list, config_list, name_list = [], [], []
results_obj = Results()

for run in runs:
    # .summary contains output keys/values for
    # metrics such as accuracy.
    #  We call ._json_dict to omit large files
    result_dict = run.summary._json_dict
    try:
        summary_dict = run.summary._json_dict
        config_dict = {k: v for k, v in run.config.items() if not k.startswith("_")}
        results_obj.fill_results(config_dict, summary_dict)
    except Exception as e:
        print(f"{e} their is a problem with {run.name} ")

print('end')


'num_states_to_perfect_fit_test' their is a problem with no_net__grammar__data_grammar_1_nguyen_2__MeasurementEncoderDummy__None__Normal__300_000 
'num_states_to_perfect_fit_test' their is a problem with no_net__grammar__data_grammar_1_nguyen_2__MeasurementEncoderDummy__None__Normal__300_000 
'num_states_to_perfect_fit_test' their is a problem with no_net__grammar__data_grammar_1_nguyen_2__MeasurementEncoderDummy__None__Normal__300_000 
'num_states_to_perfect_fit_test' their is a problem with no_net__grammar__data_grammar_1_nguyen_2__MeasurementEncoderDummy__None__Normal__300_000 
'num_states_to_perfect_fit_test' their is a problem with no_net__grammar__data_grammar_1_nguyen_2__MeasurementEncoderDummy__None__Normal__300_000 
'num_states_to_perfect_fit_test' their is a problem with no_net__grammar__data_grammar_1_nguyen_6__MeasurementEncoderDummy__None__Normal__300_000 
'num_states_to_perfect_fit_test' their is a problem with no_net__grammar__data_grammar_1_nguyen_6__MeasurementEncoderD

In [ ]:
def fill_final_result(target, approach, final_result):
    df = results_obj.result_dict[target][approach]
    nan_values = df.isna().sum().sum()
    mean_values = df.mean(axis=0).mean(axis=0).round(0)
    final_result.loc[approach][target] = mean_values
    final_result.loc[approach]['unsuccessful_fits'] = nan_values


final_result = pd.DataFrame(index=['results_uniform_1_Normal',
                                   'results_uniform_2_Normal',
                                   'results_uniform_1_AmEx',
                                   'results_uniform_2_AmEx',
                                   ],
                            columns=['num simulation to perfect fit',
                                     'num_states_to_perfect_fit_test',
                                     'time to perfect fit',
                                     'unsuccessful_fits'
                                     ]
                            )

for approch in list(final_result.index):
    for target in ['num_states_to_perfect_fit_test',
                   'time to perfect fit',
                   'num simulation to perfect fit']:
        fill_final_result(target=target,
                          approach=approch,
                          final_result=final_result)

final_result

### Average over seeds

In [30]:
from IPython.display import display, HTML
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

def highlight_min(s):
    is_min = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_min]

for target in ['num simulation to perfect fit','num_states_to_perfect_fit_test','time to perfect fit']:
    result_table = pd.DataFrame(#index=results_obj.result_dict[target].keys(),
                                columns=[f"nguyen_{i}" for i in range(1, 13, 1)])
    for approch in results_obj.result_dict[target].keys(): 
        frame = results_obj.result_dict[target][approch]
        frame_mean = frame.mean(axis=0)
        result_table.loc[approch] = frame_mean
    
    print(target)
    styled_df = result_table.style.apply(highlight_min)
    display(result_table.style.highlight_min(color = 'lightgreen',  
                       axis = 0) )

print('end')


num simulation to perfect fit


,nguyen_1,nguyen_2,nguyen_3,nguyen_4,nguyen_5,nguyen_6,nguyen_7,nguyen_8,nguyen_9,nguyen_10,nguyen_11,nguyen_12
results_grammar_1_Normal,11967.600000,nan,nan,nan,44001.800000,131521.000000,49226.400000,30155.400000,111170.600000,nan,95.800000,nan
results_uniform_1_Normal,65869.200000,107504.000000,nan,126667.000000,5016.800000,103158.800000,10814.400000,16917.000000,19048.600000,97055.250000,11.200000,58316.000000
results_grammar_1_AmEx,3567.000000,70416.000000,nan,nan,12658.200000,nan,8554.600000,11219.000000,40264.500000,nan,108.000000,nan
results_uniform_1_AmEx,3054.200000,18895.800000,88311.000000,nan,836.600000,43434.800000,2410.800000,1167.000000,3867.600000,27855.400000,9.800000,12702.200000
results_grammar_2_Normal,nan,nan,nan,nan,67101.800000,nan,10683.200000,1644.400000,25960.000000,nan,1817.800000,nan
results_uniform_2_Normal,57290.500000,9228.000000,nan,38536.000000,6359.600000,39741.000000,1008.800000,266.800000,10756.400000,51122.200000,225.600000,27194.400000
results_grammar_2_AmEx,nan,nan,nan,nan,15051.400000,nan,3657.600000,890.000000,15155.600000,70603.000000,1193.400000,31144.333333
results_uniform_2_AmEx,30675.500000,42436.000000,nan,28905.000000,1633.600000,50801.500000,974.600000,199.000000,5809.600000,19555.600000,511.000000,6263.200000


num_states_to_perfect_fit_test


,nguyen_1,nguyen_2,nguyen_3,nguyen_4,nguyen_5,nguyen_6,nguyen_7,nguyen_8,nguyen_9,nguyen_10,nguyen_11,nguyen_12
results_grammar_1_Normal,24237.200000,nan,nan,nan,77569.200000,214913.000000,86958.200000,55128.800000,181450.400000,nan,325.600000,nan
results_uniform_1_Normal,32755.600000,108768.666667,nan,312096.000000,3756.400000,48780.400000,6651.600000,10072.800000,11154.600000,92692.500000,28.200000,28987.400000
results_grammar_1_AmEx,19382.600000,358210.000000,nan,nan,65831.200000,nan,45189.400000,58850.800000,206016.500000,nan,601.200000,nan
results_uniform_1_AmEx,11428.000000,66834.200000,341128.000000,nan,2992.000000,163129.000000,8822.200000,4294.250000,14083.200000,102639.200000,28.400000,44539.600000
results_grammar_2_Normal,nan,nan,nan,nan,176079.800000,nan,32093.400000,5592.200000,73423.200000,nan,6159.600000,nan
results_uniform_2_Normal,194176.500000,34888.000000,nan,134384.000000,24274.000000,137596.000000,4290.000000,1157.400000,40134.800000,173084.000000,976.600000,95394.800000
results_grammar_2_AmEx,nan,nan,nan,nan,86498.600000,nan,22279.000000,5537.800000,87608.000000,391282.000000,7408.200000,174943.000000
results_uniform_2_AmEx,249328.250000,344338.000000,nan,236011.000000,13524.600000,409890.000000,8138.000000,1660.600000,47734.800000,160164.400000,4317.800000,51450.400000


time to perfect fit


,nguyen_1,nguyen_2,nguyen_3,nguyen_4,nguyen_5,nguyen_6,nguyen_7,nguyen_8,nguyen_9,nguyen_10,nguyen_11,nguyen_12
results_grammar_1_Normal,96.022927,nan,nan,nan,309.849375,890.285279,341.193871,222.104042,734.033578,nan,1.540657,nan
results_uniform_1_Normal,158.969036,544.796438,nan,1771.564321,17.323277,236.639215,30.125763,48.233285,51.870322,446.604099,0.102516,137.239189
results_grammar_1_AmEx,76.450980,1473.336730,nan,nan,270.178492,nan,204.929406,258.169908,913.936316,nan,2.846789,nan
results_uniform_1_AmEx,52.491553,324.510442,1969.054359,nan,13.564809,910.595396,43.687323,22.219802,71.548889,522.407096,0.120394,224.712847
results_grammar_2_Normal,nan,nan,nan,nan,654.870900,nan,126.271209,22.142734,278.736014,nan,24.585727,nan
results_uniform_2_Normal,824.531947,153.276650,nan,532.345198,104.799487,626.154170,18.104345,4.860980,181.136010,761.974252,4.387692,391.126778
results_grammar_2_AmEx,nan,nan,nan,nan,347.160602,nan,92.804515,21.670577,384.214159,1701.345130,30.874346,725.449291
results_uniform_2_AmEx,1134.432653,1623.521970,nan,1054.815129,59.333091,1963.448644,34.913578,7.485866,208.580004,742.723872,20.336101,242.168993


end
